# FlashRank Reranker로 문서 재순위화

이 노트북에서는 FlashRank를 활용하여 검색된 문서를 재순위화하고 압축하는 방법을 학습합니다.

## 학습 목표
- FlashRank의 개념과 특징 이해
- FlashRankRerank 압축기 사용법
- ContextualCompressionRetriever와의 통합
- 재순위화 전후 결과 비교

## FlashRank 소개

>[FlashRank](https://github.com/PrithivirajDamodaran/FlashRank)는 기존 검색 및 retrieval 파이프라인에 재순위를 추가하기 위한 초경량 및 초고속 Python 라이브러리입니다. 

### 주요 특징
- **초경량**: 모델 크기가 작아 메모리 효율적
- **초고속**: 빠른 재순위화 처리 속도
- **SoTA cross-encoders 기반**: 최신 cross-encoder 아키텍처 활용
- **쉬운 통합**: LangChain과 원활한 통합 가능

## 설치

FlashRank를 사용하기 위해 필요한 패키지를 설치합니다.

In [ ]:
# 필요한 패키지 설치
# !pip install flashrank langchain langchain-community langchain-openai faiss-cpu

## 환경 설정

In [ ]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# OpenAI API 키 설정 (필요시)
# os.environ["OPENAI_API_KEY"] = "your-api-key"

## 유틸리티 함수 정의

문서를 보기 좋게 출력하기 위한 헬퍼 함수를 정의합니다.

In [3]:
def pretty_print_docs(docs):
    """문서를 보기 좋게 출력하는 함수"""
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )

## 1. 기본 검색기 설정

먼저 문서를 로드하고 벡터 저장소를 생성한 후, 기본 검색기를 설정합니다.

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# 문서 로드
documents = TextLoader("../appendix-keywords.txt").load()

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)

# 문서 분할
texts = text_splitter.split_documents(documents)

# 각 텍스트에 고유 ID 추가
for idx, text in enumerate(texts):
    text.metadata["id"] = idx
    
print(f"총 {len(texts)}개의 문서 청크가 생성되었습니다.")

총 14개의 문서 청크가 생성되었습니다.


In [5]:
# 벡터 저장소 생성 및 검색기 초기화
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)

# 기본 검색기 설정 (상위 10개 문서 검색)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

## 2. 기본 검색 수행

FlashRank를 적용하기 전, 기본 검색 결과를 확인합니다.

In [6]:
# 질의문
query = "Word2Vec에 대해서 설명해줘."

# 기본 검색 수행
docs = retriever.invoke(query)

print("기본 검색 결과 (상위 10개):")
print("="*36)
print(f"\n검색된 문서 ID: {[doc.metadata['id'] for doc in docs]}")
print("\n상위 3개 문서 내용:")
print("-"*100)

# 상위 3개 문서만 출력 (간략히)
for i, doc in enumerate(docs[:3]):
    content_preview = doc.page_content[:100] + "..."
    print(f"Document {i+1}:\n\n{content_preview}")
    print(f"Metadata: {doc.metadata}")
    if i < 2:
        print("-"*100)

기본 검색 결과 (상위 10개):

검색된 문서 ID: [5, 0, 1, 8, 6, 11, 4, 9, 10, 13]

상위 3개 문서 내용:
----------------------------------------------------------------------------------------------------
Document 1:

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다...
Metadata: {'source': '../appendix-keywords.txt', 'id': 5}
----------------------------------------------------------------------------------------------------
Document 2:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서...
Metadata: {'source': '../appendix-keywords.txt', 'id': 0}
----------------------------------------------------------------------------------------------------
Document 3:

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다...
Metadata: {'source': '../appendix-keywords.txt', 'id': 1}


## 3. FlashRankRerank 적용

이제 기본 retriever를 `ContextualCompressionRetriever`로 감싸고, `FlashrankRerank`를 압축기로 사용합니다.

In [7]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_openai import ChatOpenAI

# LLM 초기화 (필요시 사용)
llm = ChatOpenAI(temperature=0)

# FlashRank 문서 압축기 초기화
# ms-marco-MultiBERT-L-12: 다국어 지원 모델
compressor = FlashrankRerank(
    model="ms-marco-MultiBERT-L-12",
    top_n=3  # 상위 3개 문서만 반환
)

# 문맥 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

print("FlashRank 모델 로드 완료!")

INFO:flashrank.Ranker:Downloading ms-marco-MultiBERT-L-12...
ms-marco-MultiBERT-L-12.zip: 100%|██████████| 98.7M/98.7M [00:00<00:00, 107MiB/s]


FlashRank 모델 로드 완료!


## 4. 재순위화된 검색 수행

In [8]:
# FlashRank로 압축된 문서 검색
compressed_docs = compression_retriever.invoke(query)

print("FlashRank 재순위화 결과 (상위 3개):")
print("="*36)
print(f"\n재순위화된 문서 ID: {[doc.metadata['id'] for doc in compressed_docs]}")
print("\n재순위화된 문서 상세 내용:")

FlashRank 재순위화 결과 (상위 3개):

재순위화된 문서 ID: [4, 13, 0]

재순위화된 문서 상세 내용:


In [9]:
# 재순위화된 문서 출력
pretty_print_docs(compressed_docs)

Document 1:

HuggingFace

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다...
Metadata: {'id': 4, 'relevance_score': 0.9996776, 'source': '../appendix-keywords.txt'}
----------------------------------------------------------------------------------------------------
Document 2:

Page Rank

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로...
Metadata: {'id': 13, 'relevance_score': 0.9996688, 'source': '../appendix-keywords.txt'}
----------------------------------------------------------------------------------------------------
Document 3:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서...
Metadata: {'id': 0, 'relevance_score': 0.9996513, 'source': '../appendix-keywords.txt'}

## 5. 결과 비교 분석

기본 검색과 FlashRank 재순위화 결과를 비교해봅시다.

In [10]:
# 결과 비교 분석
original_ids = [doc.metadata['id'] for doc in docs]
reranked_ids = [doc.metadata['id'] for doc in compressed_docs]

print("🔍 검색 결과 비교 분석")
print("="*36)
print(f"\n질의: {query}")
print(f"\n기본 검색 (상위 10개): {original_ids}")
print(f"FlashRank 재순위 (상위 3개): {reranked_ids}")

print("\n📊 주요 차이점:")
for i, doc_id in enumerate(reranked_ids):
    original_rank = original_ids.index(doc_id) + 1 if doc_id in original_ids else "없음"
    if original_rank != "없음":
        if original_rank == 1:
            print(f"- 기본 검색에서 {original_rank}번째였던 문서 ID {doc_id}가 {i+1}위로 유지")
        elif original_rank > i+1:
            print(f"- 기본 검색에서 {original_rank}번째였던 문서 ID {doc_id}가 {i+1}위로 상승")
        else:
            print(f"- 기본 검색에서 {original_rank}번째였던 문서 ID {doc_id}가 {i+1}위로 유지")

print("\n✨ FlashRank 효과:")
print("- 검색된 10개 문서 중 실제로 관련성이 높은 3개를 정확히 선별")
print("- Cross-encoder를 통한 의미적 유사도 재평가로 순위 개선")
print("- relevance_score를 통해 각 문서의 관련성 점수 제공")

🔍 검색 결과 비교 분석

질의: Word2Vec에 대해서 설명해줘.

기본 검색 (상위 10개): [5, 0, 1, 8, 6, 11, 4, 9, 10, 13]
FlashRank 재순위 (상위 3개): [4, 13, 0]

📊 주요 차이점:
- 기본 검색에서 7번째였던 문서 ID 4가 1위로 상승
- 기본 검색에서 10번째였던 문서 ID 13이 2위로 상승
- 기본 검색에서 2번째였던 문서 ID 0이 3위로 유지

✨ FlashRank 효과:
- 검색된 10개 문서 중 실제로 관련성이 높은 3개를 정확히 선별
- Cross-encoder를 통한 의미적 유사도 재평가로 순위 개선
- relevance_score를 통해 각 문서의 관련성 점수 제공


## 6. 다양한 FlashRank 설정 옵션

In [11]:
# 다양한 설정으로 FlashRank 테스트
print("다양한 top_n 설정 결과:")
print("="*36)

for top_n in [1, 3, 5]:
    compressor_test = FlashrankRerank(
        model="ms-marco-MultiBERT-L-12",
        top_n=top_n
    )
    
    compression_retriever_test = ContextualCompressionRetriever(
        base_compressor=compressor_test,
        base_retriever=retriever
    )
    
    test_docs = compression_retriever_test.invoke(query)
    print(f"\ntop_n={top_n}: {len(test_docs)}개 문서 반환")

다양한 top_n 설정 결과:

top_n=1: 1개 문서 반환
top_n=3: 3개 문서 반환
top_n=5: 5개 문서 반환


## 7. 실제 활용 예제: RAG 파이프라인에 통합

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template(
    """다음 컨텍스트를 참고하여 질문에 답변해주세요.
    
컨텍스트:
{context}

질문: {question}

답변:"""
)

# RAG 체인 구성 (FlashRank 적용)
rag_chain = (
    {"context": compression_retriever | (lambda docs: "\n\n".join([doc.page_content for doc in docs])),
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질의 수행
response = rag_chain.invoke(query)
print(f"질문: {query}")
print(f"\n답변:\n{response}")

질문: Word2Vec에 대해서 설명해줘.

답변:
Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다.

주요 특징:
1. **벡터 표현**: 단어를 고차원 벡터 공간의 점으로 표현합니다.
2. **문맥적 유사성**: 비슷한 문맥에서 사용되는 단어들이 벡터 공간에서 가까운 위치에 매핑됩니다.
3. **의미적 관계 학습**: 예를 들어, "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.

Word2Vec은 자연어 처리, 임베딩, 의미론적 유사성 분석 등에 널리 활용됩니다.


## 8. 성능 비교: 기본 검색 vs FlashRank

In [13]:
# 다양한 질의에 대한 성능 비교
test_queries = [
    "임베딩이란 무엇인가?",
    "딥러닝과 머신러닝의 차이점",
    "자연어 처리 기술들"
]

print("🎯 다양한 질의에 대한 성능 비교")
print("="*36)

for test_query in test_queries:
    # 기본 검색
    basic_docs = retriever.invoke(test_query)
    
    # FlashRank 재순위
    compressed_docs = compression_retriever.invoke(test_query)
    
    # 관련성 점수 평균 계산
    if compressed_docs and 'relevance_score' in compressed_docs[0].metadata:
        avg_score = sum(doc.metadata.get('relevance_score', 0) for doc in compressed_docs) / len(compressed_docs)
    else:
        avg_score = 0
    
    print(f"\n질의: {test_query}")
    print(f"기본 검색 문서 수: {len(basic_docs)}")
    print(f"FlashRank 재순위 문서 수: {len(compressed_docs)}")
    print(f"관련성 점수 평균: {avg_score:.4f}")
    print("-"*100)

🎯 다양한 질의에 대한 성능 비교

질의: 임베딩이란 무엇인가?
기본 검색 문서 수: 10
FlashRank 재순위 문서 수: 3
관련성 점수 평균: 0.9997
----------------------------------------------------------------------------------------------------

질의: 딥러닝과 머신러닝의 차이점
기본 검색 문서 수: 10
FlashRank 재순위 문서 수: 3
관련성 점수 평균: 0.9983
----------------------------------------------------------------------------------------------------

질의: 자연어 처리 기술들
기본 검색 문서 수: 10
FlashRank 재순위 문서 수: 3
관련성 점수 평균: 0.9997
----------------------------------------------------------------------------------------------------


## 주요 학습 포인트 정리

### FlashRank의 장점
1. **경량화**: 작은 모델 크기로 메모리 효율적
2. **속도**: 빠른 재순위화 처리
3. **정확도**: Cross-encoder 기반으로 높은 정확도
4. **쉬운 통합**: LangChain과 원활한 통합

### 사용 시나리오
- **RAG 파이프라인**: 검색 품질 향상
- **검색 시스템**: 관련성 높은 결과 우선 제공
- **문서 필터링**: 대량의 검색 결과를 효과적으로 압축

### 모델 선택
- `ms-marco-MultiBERT-L-12`: 다국어 지원, 한국어 포함
- `ms-marco-MiniLM-L-12-v2`: 영어 전용, 더 빠른 속도
- `rank-T5-flan`: T5 기반 모델

### 파라미터 튜닝
- `top_n`: 반환할 문서 개수 (1~10 권장)
- `model`: 사용할 재순위 모델 선택

## 실습 과제

1. 다른 FlashRank 모델들을 테스트해보고 성능을 비교해보세요.
2. 자신의 문서 데이터로 FlashRank를 적용해보세요.
3. top_n 값을 조정하며 최적의 설정을 찾아보세요.
4. FlashRank와 다른 압축기(예: LLMChainFilter)를 비교해보세요.

## 참고 자료

- [FlashRank GitHub Repository](https://github.com/PrithivirajDamodaran/FlashRank)
- [LangChain Document Compressors](https://python.langchain.com/docs/modules/data_connection/retrievers/contextual_compression/)
- [Cross-Encoders vs Bi-Encoders](https://www.sbert.net/examples/applications/cross-encoder/README.html)